https://www.kaggle.com/clair14/lgbm-bayesianoptimization

In [13]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random
import os
from sklearn.metrics import log_loss
from bayes_opt import BayesianOptimization
import lightgbm

train = pd.read_csv('../input/cc123123123/train.csv')
test = pd.read_csv('../input/cc123123123/test.csv')
ss = pd.read_csv('../input/cc123123123/sample_submission.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

# Married, Civil marriage
train['income_per_size'] = np.log(train['income_total']/train['family_size'])
test['income_per_size'] = np.log(test['income_total']/test['family_size'])
train.loc[(train['family_type']=='Married')|(train['family_type']=='Civil marriage'),'income_per_size']\
= train['income_per_size'] * 2

test.loc[(test['family_type']=='Married')|(test['family_type']=='Civil marriage'),'income_per_size']\
= test['income_per_size'] * 2

def simple_marry(x):
    if x == 'Married' or x =='Civil marriage':
        return '0'
    elif x == 'Separated' or x == 'Widow':
        return '1'
    else:
        return '2'

for df in [train,test]:
    df['family_bins'] = df['family_type'].apply(simple_marry)

# income_total을 로그변환
train['income_total'] = np.log(train['income_total'])
# train = train.drop('income_total',1)
test['income_total'] = np.log(test['income_total'])
# test = test.drop('income_total',1)

# car와 reality를 합친 새로운 칼럼 careality
train['car'] =train['car'].apply(lambda x: int(x=='Y'))
train['reality'] =train['reality'].apply(lambda x: int(x=='Y'))
test['car'] =test['car'].apply(lambda x: int(x=='Y'))
test['reality'] =test['reality'].apply(lambda x: int(x=='Y'))

train['careality'] = train['car'] + train['reality']
train = train.drop(['car', 'reality'],1)

test['careality'] = test['car'] + test['reality']
test = test.drop(['car', 'reality'],1)

object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

## 제곱
for df in [train,test]:
    df['income_per_size'] = df['income_per_size'].apply(lambda x: x**2)
    
c = 'income_per_size'
mean = train[c].mean()
std = train[c].std()

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'income_total'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'begin_month'
k = 2.2

train[c] = train[c].apply(lambda x: x**2)
test[c] = test[c].apply(lambda x: x**2)

mean = train[c].mean()
std = train[c].std()

idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
# train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

######################
c = 'DAYS_BIRTH'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

#기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std



##########################
# out: 아웃라이어
out_train = train.loc[train.DAYS_EMPLOYED>0]
out_test = test.loc[test.DAYS_EMPLOYED>0]

in_train = train.loc[train.DAYS_EMPLOYED<=0]
in_test = test.loc[test.DAYS_EMPLOYED<=0]




###################
c = 'DAYS_EMPLOYED'
k = 2.2
mean = in_train[c].mean()
std = in_train[c].std()
idxs = in_train.loc[(in_train[c]>= mean + k*std)|\
                (in_train[c]<= mean - k*std)].index
in_train = in_train.drop(idxs).reset_index(drop=True)

# 기존 
in_train[c] = (in_train[c]-mean)/std
in_test[c] = (in_test[c]-mean)/std

# #######모델링, 학습 시작#######

|365243?|학습|예측|
|----|----|----|
|yes|out_train|out_test|
|no|in_train|in_test|

In [14]:
# !pip install bayesian-optimization

In [19]:
temp = in_train
def lgb_eval(n_estimators, num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf, learning_rate):
    

    params = {
        "objective" : "multiclass",
        "metric" : "multi_logloss", 
        "num_classes" : 3,
        'is_unbalance': True,
        'n_estimators': int(n_estimators),
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : learning_rate,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(temp.drop('credit',1), temp['credit'])
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=10)
    return -cv_result['multi_logloss-mean'][-1]

In [ ]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                        'n_estimators': (300,2000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000),
                                                'learning_rate': (0.01, 0.3)
                                                })

lgbBO.maximize(n_iter=30, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_da... | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------------------


In [18]:
lgbBO.max

{'target': -0.7666262326076231,
 'params': {'lambda_l1': 0.04619401127783732,
  'lambda_l2': 0.023516340914865916,
  'learning_rate': 0.21142050117535985,
  'max_depth': 9.817818031569452,
  'min_child_samples': 1069.5789469296615,
  'min_data_in_leaf': 159.20038594955219,
  'n_estimators': 1440.4418354272257,
  'num_leaves': 3976.0290566566396}}

# #######모델링, 학습 끝#######

In [ ]:
in_zeros = np.zeros([len(in_test),3])
for fold in range(n):
    in_zeros += in_models[fold].predict_proba(in_test)/n
in_output = pd.DataFrame(in_zeros)
in_output = in_output.reindex(index=pd.Index(idx_in_test))

In [ ]:
out_zeros = np.zeros([len(out_test),3])
for fold in range(n):
    out_zeros += out_models[fold].predict_proba(out_test)/n
out_output = pd.DataFrame(out_zeros)
out_output = out_output.reindex(index=pd.Index(idx_out_test))

In [ ]:
in2 = np.concatenate((in_zeros,np.array(idx_in_test).reshape(-1,1)),axis=1)
out2 = np.concatenate((out_zeros,np.array(idx_out_test).reshape(-1,1)),axis=1)

In [ ]:
output = pd.DataFrame(np.concatenate((in2,out2),axis=0),columns=[0,1,2,'index'])
output['index'] = output['index'].astype('int')

In [ ]:
for i in range(len(output)):
    row = output.loc[output['index']==i]
    ss.iloc[i,1:] = row.iloc[:,:3]

In [ ]:
ss.to_csv('two_models.csv', index=False)